# 04 – Dev-Abschlusslauf: Finale Konfiguration (min_on=1.0, min_off=1.2, beam=12, len=20)

## Kontext

Dieses Notebook ist der **finale Dev-Durchlauf** mit der in `02k_` identifizierten
optimalen Parameterkombination:

| Parameter | Wert | Quelle |
|-----------|------|--------|
| `beam_size` | 12 | `02_` + `02j_` Grid-Search (Decoding) |
| `max_length` | 20 s | `02_` + `02j_` Grid-Search (Decoding) |
| `min_duration_on` | 1.0 s | `02k_` Grid-Search (E72) |
| `min_duration_off` | 1.2 s | `02k_` Grid-Search (E72) |
| Modell | BL4 (`avsr_cocktail_finetuned`) | Baseline |

## Ablauf

1. Segmentierungs-Monkey-Patch (aus `02k_`) aktiv setzen
2. Inference auf allen Dev-Sessions
3. Evaluation + Vergleich mit BL4-Baseline

## Ergebnis (Vorschau)

| Konfiguration | WER | Conv F1 | Joint Error | Δ WER vs. BL4 |
|--------------|-----|---------|-------------|---------------|
| BL4-Baseline (beam=3, len=15) | 0.4987 | 0.8153 | 0.3547 | – |
| **Dev-Final (E72-Konfig, 25 Sessions)** | **0.4943** | **0.8553** | **0.3318** | **-0.0044 (-0.88%)** |

Bestes Ergebnis der gesamten Experimentreihe auf dem Dev-Set.

## 1 – GPU-Check

In [1]:
!nvidia-smi

Thu Feb  5 04:52:48 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   33C    P0             80W /  500W |    9557MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## 2 – CUDA-Verifikation

In [2]:
import os

# Physische GPU-Auswahl: hier GPU 2 (siehe nvidia-smi)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [3]:
import torch, gc, glob
from pathlib import Path
import pandas as pd

In [4]:
print("CUDA devices:", torch.cuda.device_count())
print("Device 0 name (sollte A100 sein):", torch.cuda.get_device_name(0))

CUDA devices: 1
Device 0 name (sollte A100 sein): NVIDIA A100-SXM4-80GB


## 3 – Setup: Arbeitsverzeichnis & Imports

In [5]:
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

import sys
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, append_eval_results_for_experiments

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 4 – Segmentierungs-Patch (aus `02k_`)

Identisch zu `02k_`: Context-Manager überschreibt `InferenceEngine.chunk_video`
temporär, damit `min_duration_on` und `min_duration_off` an `segment_by_asd()`
weitergegeben werden. Nach dem `with`-Block wird die Original-Methode wiederhergestellt.

In [6]:
from contextlib import contextmanager
import script.inference as inf  # das ist das Modul, das pg_utils_experiments nutzt

@contextmanager
def patch_avsr_segmentation(min_on=None, min_off=None):
    # Patcht InferenceEngine.chunk_video für den ASD-Zweig
    # Wird als Context-Manager verwendet; Original wird in finally wiederhergestellt
    orig = inf.InferenceEngine.chunk_video

    if min_on is None and min_off is None:
        yield
        return

    def patched(self, video_path, asd_path=None, max_length=15):
        if asd_path is not None:
            with open(asd_path, "r") as f:
                asd = inf.json.load(f)

            frames = sorted([int(f) for f in asd.keys()])
            if not frames:
                return []
            min_frame = min(frames)

            params = {"max_chunk_size": max_length}
            if min_on is not None:
                params["min_duration_on"] = float(min_on)
            if min_off is not None:
                params["min_duration_off"] = float(min_off)

            segments_by_frames = inf.segment_by_asd(asd, params)
            return [((seg[0] - min_frame) / 25, (seg[-1] - min_frame) / 25) for seg in segments_by_frames]

        return orig(self, video_path, asd_path, max_length)

    inf.InferenceEngine.chunk_video = patched
    try:
        yield
    finally:
        inf.InferenceEngine.chunk_video = orig

def _tag(x: float) -> str:
    # Float → Dateinamen-sicherer String: 1.2 → '1p2'
    return str(x).replace(".", "p")

## 5 – Split-Konfiguration

`SPLIT = "DEV"` → `data-bin/dev`. Dasselbe Notebook kann für EVAL genutzt werden
(nur `SPLIT` ändern) – `04_eval_final_results` tut genau das.

In [7]:
SPLIT = "DEV"
DATA_ROOT = "data-bin/dev" if SPLIT == "DEV" else "data-bin/eval"  # ggf. anpassen

## 6 – Experiment-Definition

Automatisch generierter `exp_name` kodiert alle relevanten Parameter:
`DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20`.
Nur BL4 (`avsr_cocktail_finetuned`) im `BASE_MODELS`-Dict, da ausschließlich dieses Modell genutzt wird.

In [8]:
beam_size  = 12
max_length = 20
best_min_on  = 1.0 # Beste Konfiguration aus 02j_ (E72)
best_min_off = 1.2 # Beste Konfiguration aus 02j_ (E72)

# exp_name kodiert alle Parameter im Verzeichnisnamen
exp_name = f"{SPLIT}_final_bugfix_mdOn{_tag(best_min_on)}_mdOff{_tag(best_min_off)}_bs{beam_size}_len{max_length}"
BASE_MODELS = {
    "avsr_cocktail_finetuned": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
    },
}
EXPERIMENTS = {
    exp_name: {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": beam_size,
        "max_length": max_length,
        "comment": f"{SPLIT} FINAL: AVSR override min_on={best_min_on}, min_off={best_min_off}",
    }
}

## 7 – Session-Erkennung

`metadata.json`-Filter stellt sicher, dass nur vollständig vorbereitete
Sessions verarbeitet werden (keine leeren oder unvollständigen Verzeichnisse).

In [12]:
# Sessions für den Split finden
session_dirs = sorted([p for p in glob.glob(f"{DATA_ROOT}/session_*")
                       if Path(p, "metadata.json").exists()])
session_ids = [Path(p).name for p in session_dirs]
print("Split:", SPLIT, "| Sessions:", len(session_ids))
print(session_ids)

Split: DEV | Sessions: 25
['session_132', 'session_133', 'session_134', 'session_135', 'session_136', 'session_137', 'session_138', 'session_139', 'session_140', 'session_141', 'session_40', 'session_41', 'session_42', 'session_43', 'session_44', 'session_48', 'session_49', 'session_50', 'session_51', 'session_52', 'session_53', 'session_54', 'session_55', 'session_56', 'session_57']


## 8 – Inference

Jede Session wird innerhalb des Monkey-Patch-Context-Managers verarbeitet.
CUDA-Cache und GC werden nach jeder Session geleert.

In [13]:
# ======= Inference pro Session =======
for session_dir in session_dirs:
    print("\nRUN:", SPLIT, Path(session_dir).name)
    with patch_avsr_segmentation(min_on=best_min_on, min_off=best_min_off):
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=BASE_MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )
    torch.cuda.empty_cache() # GPU-Speicher nach jeder Session freigeben
    gc.collect()


RUN: DEV session_132

Starte Inference für Experiment: DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 12
  max_length      = 20
  output_dir_name = output_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
  session_dir     = data-bin/dev/session_132
  comment         = DEV FINAL: AVSR override min_on=1.0, min_off=1.2
Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_132


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [01:46<08:53, 106.68s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 21/21 [01:12<00:00,  3.47s/it]

Processing speakers:  20%|██        | 1/5 [01:34<06:17, 94.29s/it]


Processing speakers:  40%|████      | 2/5 [02:17<03:13, 64.48s/it]


Processing speakers:  60%|██████    | 3/5 [02:53<01:42, 51.24s/it]


Processing speaker spk_3 track 1 of 2: 100%|██████████| 21/21 [00:27<00:00,  1.30s/it]

Processing speakers:  80%|████████  | 4/5 [03:50<00:53, 53.66s/it]


Processing speakers: 100%|██████████| 5/5 [05:10<00:00, 62.19s/it]


## 9 – Evaluation & Aggregation

In [15]:
# ======= Evaluation + CSV =======
df_by_session = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=session_ids,
    target_csv=f"results_{exp_name}_by_session.csv",
    session_dir_template=f"{DATA_ROOT}" + "/{sid}",
    label_dir="labels",
)

summary = (df_by_session
           .groupby("model", as_index=False)[["avg_conv_f1","avg_speaker_wer","avg_joint_error"]]
           .mean())

summary.to_csv(f"results_{exp_name}_summary.csv", index=False)
display(summary)
print("Saved:", f"results_{exp_name}_by_session.csv", f"results_{exp_name}_summary.csv")


########## Evaluate für session_132 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev/session_132 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_132 ===

--- Evaluating output dir: output_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.3473, 'spk_1': 0.5906, 'spk_2': 0.4851, 'spk_3': 0.4842, 'spk_4': 0.6405, 'spk_5': 0.6696}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.17365, 'spk_1': 0.2953, 'spk_2': 0.24255, 'spk_3': 0.2421, 'spk_4': 0.32025, 'spk_5': 0.3348}

--- Evaluating output dir: output_auto_avsr ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.8698, 'spk_1': 0.8679, 'spk_2': 0.9183, 'spk_3': 0.8807, 'spk_4': 0.8512, 'spk_5': 0.8877}
Speaker clus

/home/josch080/Projektgruppe/mcorec_baseline/script/pg_utils_experiments.py:381: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_df], ignore_index=True)


,model,avg_conv_f1,avg_speaker_wer,avg_joint_error
0,DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20,0.855282,0.494316,0.331767


Saved: results_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20_by_session.csv results_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20_summary.csv


## 10 – Ergebnisanalyse: Dev-Final vs. BL4-Baseline

Robuste CSV-Suche mit Fallback (falls `exp_name` nicht gesetzt).
BL4-Identifikation unterstützt beide möglichen Spaltennamen (`model` und `exp`).

In [16]:
import glob
import pandas as pd

# Run-CSV laden
# Wenn exp_name im Notebook noch definiert ist:
run_csv = f"results_{exp_name}_by_session.csv"

# Fallback: falls exp_name nicht existiert, nimm die neueste results_*_by_session.csv
if "exp_name" not in globals():
    cands = sorted(glob.glob("results_*_by_session.csv"))
    if not cands:
        raise FileNotFoundError("Keine results_*_by_session.csv gefunden.")
    run_csv = cands[-1]
    print("exp_name nicht gesetzt -> verwende:", run_csv)
else:
    print("Verwende:", run_csv)

df_by_session = pd.read_csv(run_csv)

# Summary: Mittelwert über alle Dev-Sessions
metrics = ["avg_conv_f1","avg_speaker_wer","avg_joint_error"]

summary = (df_by_session
           .groupby("model", as_index=False)[metrics]
           .mean()
           .sort_values("avg_speaker_wer"))

summary_csv = run_csv.replace("_by_session.csv", "_summary.csv")
summary.to_csv(summary_csv, index=False)

display(summary)
print("Saved:", run_csv, summary_csv)

# BL4 Baseline laden
baseline_cands = []
baseline_cands += glob.glob("results_baseline_dev_without_central_videos*.csv")
baseline_cands += glob.glob("/mnt/data/results_baseline_dev_without_central_videos*.csv")

if not baseline_cands:
    raise FileNotFoundError("Baseline CSV nicht gefunden: results_baseline_dev_without_central_videos*.csv")

baseline_csv = sorted(baseline_cands)[0]
print("Baseline CSV:", baseline_csv)

df_base = pd.read_csv(baseline_csv)

# BL4 in beiden möglichen Spaltenformaten suchen
mask = pd.Series([False] * len(df_base))
if "model" in df_base.columns:
    mask |= df_base["model"].astype(str).eq("avsr_cocktail_finetuned")
if "exp" in df_base.columns:
    mask |= df_base["exp"].astype(str).eq("output_avsr_cocktail_finetuned")

bl4 = df_base[mask]
if bl4.empty:
    raise ValueError("BL4 nicht gefunden. Erwartet model='avsr_cocktail_finetuned' oder exp='output_avsr_cocktail_finetuned'.")
bl4 = bl4.iloc[0]

# Vergleichstabelle: Dev-Final vs. BL4
# delta_wer: negativ = Verbesserung (Run-WER < BL4-WER)
run_row = summary.iloc[0]

compare = pd.DataFrame([{
    "run_model": run_row["model"],
    "run_avg_conv_f1": float(run_row["avg_conv_f1"]),
    "run_avg_speaker_wer": float(run_row["avg_speaker_wer"]),
    "run_avg_joint_error": float(run_row["avg_joint_error"]),

    "bl4_avg_conv_f1": float(bl4["avg_conv_f1"]),
    "bl4_avg_speaker_wer": float(bl4["avg_speaker_wer"]),
    "bl4_avg_joint_error": float(bl4["avg_joint_error"]),

    "delta_wer_abs": float(run_row["avg_speaker_wer"] - bl4["avg_speaker_wer"]),
    "delta_wer_rel_%": float((run_row["avg_speaker_wer"] - bl4["avg_speaker_wer"]) / bl4["avg_speaker_wer"] * 100.0),

    "delta_joint_abs": float(run_row["avg_joint_error"] - bl4["avg_joint_error"]),
    "delta_joint_rel_%": float((run_row["avg_joint_error"] - bl4["avg_joint_error"]) / bl4["avg_joint_error"] * 100.0),
}])

compare_csv = run_csv.replace("_by_session.csv", "_vs_BL4.csv")
compare.to_csv(compare_csv, index=False)

display(compare)
print("Saved:", compare_csv)

print("\nVergleich Run vs. BL4:")
print(f"  Run: {compare.loc[0,'run_model']}")
print(f"    WER={compare.loc[0,'run_avg_speaker_wer']:.6f} | Joint={compare.loc[0,'run_avg_joint_error']:.6f}")
print(f"  BL4: avsr_cocktail_finetuned")
print(f"    WER={compare.loc[0,'bl4_avg_speaker_wer']:.6f} | Joint={compare.loc[0,'bl4_avg_joint_error']:.6f}")
print(f"  ΔWER   {compare.loc[0,'delta_wer_abs']:+.6f}  ({compare.loc[0,'delta_wer_rel_%']:+.3f}%)")
print(f"  ΔJoint {compare.loc[0,'delta_joint_abs']:+.6f}  ({compare.loc[0,'delta_joint_rel_%']:+.3f}%)")


Verwende: results_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20_by_session.csv


,model,avg_conv_f1,avg_speaker_wer,avg_joint_error
0,DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20,0.855282,0.494316,0.331767


Saved: results_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20_by_session.csv results_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20_summary.csv
Baseline CSV: results_baseline_dev_without_central_videos.csv


,run_model,run_avg_conv_f1,run_avg_speaker_wer,run_avg_joint_error,bl4_avg_conv_f1,bl4_avg_speaker_wer,bl4_avg_joint_error,delta_wer_abs,delta_wer_rel_%,delta_joint_abs,delta_joint_rel_%
0,DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20,0.855282,0.494316,0.331767,0.815282,0.498727,0.354674,-0.004412,-0.884586,-0.022906,-6.458398


Saved: results_DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20_vs_BL4.csv

Vergleich Run vs. BL4:
  Run: DEV_final_bugfix_mdOn1p0_mdOff1p2_bs12_len20
    WER=0.494316 | Joint=0.331767
  BL4: avsr_cocktail_finetuned
    WER=0.498727 | Joint=0.354674
  ΔWER   -0.004412  (-0.885%)
  ΔJoint -0.022906  (-6.458%)


## 11 – Interpretation

Der Dev-Abschlusslauf mit der aus dem Grid-Search ermittelten Konfiguration
(E72: min_on=1.0 s, min_off=1.2 s, beam=12, len=20) erzielt das
**beste Ergebnis der gesamten Experimentreihe** auf dem Dev-Set.

Die gleichen Parameter werden in `04_eval_final_results` auf das
**Eval-Set** (unbekannte Daten) angewendet, das für die Einreichung genutzt wird.

## Fazit: Finale Ergebnisse auf dem Dev-Set (25 Sessions)

| Metrik | BL4-Baseline | Dev-Final | Δ absolut | Δ relativ |
|--------|-------------|-----------|-----------|----------|
| Speaker WER (↓) | 0.4987 | 0.4943 | -0.0044 | -0.88% |
| Conv F1 (↑) | 0.8153 | 0.8553 | +0.0400 | +4.91% |
| Joint Error (↓) | 0.3547 | 0.3318 | -0.0229 | -6.46% |